# Global Average
* Outputs the mean score across all (user, item) pairs
  * Prediction for user $i$ and item $j$ is $\tilde r_{ij} = a$
  * Loss function is $L = \sum_\Omega(r_{ij} - \tilde r)^2$
  * $\Omega$ is the set of oberved pairs $(i, j)$
  * $r_{ij}$ is the rating for user $i$ and item $j$
* This is a useful baseline to measure improvements from

In [2]:
name = "GlobalAverage";
residual_alphas = [];

In [1]:
using NBInclude
@nbinclude("Alpha.ipynb");

## Training

In [3]:
μ = mean(training.my_score);

## Inference

In [4]:
function make_prediction(users, items, μ)
    return fill(μ, length(users))
end;

model(users, items) = make_prediction(users, items, μ);

In [5]:
write_predictions(model);

training set: RMSE 1.7805202512279754 MAE 1.3906332220653705 R2 -8.201576149814206e28
validation set: RMSE 1.7806075310542464 MAE 1.3908660830061252 R2 -1.1164350882614358e29


In [6]:
write_params(Dict("μ" => μ));